# Debugging Float64 exponentiation

Several formally correct inequalities do not hold numerically in Julia at the time of this writing. This notebook isolates at least one aspect of the problem, the `Float64` exponentiation method. 

According to [Effective Bounds 2](http://michaelnielsen.org/polymath1/index.php?title=Effective_bounds_on_H_t_-_second_approach), "We use O≤(X) to denote any quantity bounded in magnitude by at most X. An equality A=B using this notation means that any quantity of the form A is also of the form B, though we do not require the converse to be true (thus we break the symmetry of the equality relation). Thus for instance O≤(1)+O≤(1)=O≤(3)."

Elementary estimate (v) is:

$\Gamma(z) = \sqrt{2\pi} \exp( (z-\frac{1}{2}) \log z - z + O_{\leq}( \frac{1}{12(|z| - 0.33)} )).$

This suggests something like this to me:

$\Gamma(z) = \sqrt{2\pi} \exp( (z-\frac{1}{2}) \log z - z \pm O_{\leq}( \frac{1}{12(|z| - 0.33)} )),$

*i.e.* that $\Gamma(z)$ is within a factor $\exp(\pm O_{\leq}( \frac{1}{12(|z| - 0.33)} )$ of $\sqrt{2\pi} \exp( (z-\frac{1}{2}) \log z - z),$ or

$$\exp( \frac{-1}{12(|z| - 0.33)} ) \le |\frac{\Gamma(z)}{\sqrt{2\pi} \exp( (z-\frac{1}{2}) \log z - z)}| \le \exp( \frac{1}{12(|z| - 0.33)} )$$


In [3]:
using DBNUpperBound
using SpecialFunctions

In [4]:
function est5(z::Number)
    return √(2*π)*bigexp((z-0.5)*log(z)-z),bigexp(1/(12*(abs(z)-0.33)))
end

est5 (generic function with 1 method)

In [5]:
setprecision(80)do
    @show G = bigexp(lgamma(300.0+.4im))
    @show est = est5(300.0+.4im)
    @show 1/est[2] ≤ abs(G/est[1]) ≤ est[2]
    nothing
end

G = bigexp(lgamma(300.0 + 0.4im)) = -6.6485627886914067585544807e+611 + 7.7343479502546049890029821e+611im
est = est5(300.0 + 0.4im) = (-6.6467190899089837930129941e+611 + 7.732197361507379971825832e+611im, 1.0002781220911363)
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true


In [6]:
srand(0x1234)
r5 = region_5(1000)
tmp = Vector{Bool}(size(r5,1))
for i in 1:size(r5,1)
    s = s⁺(r5[i,2],r5[i,3])
    G = bigexp(lgamma(s))
    est = est5(s)
    tmp[i] = 1/est[2] ≤ abs(G/est[1]) ≤ est[2]
end
@show all(tmp)
cases = r5[.!tmp,2:3]
@show size(cases,1)
nothing

all(tmp) = false
size(cases, 1) = 4


In [7]:
cases

4×2 Array{Float64,2}:
 944.963  0.611094
 947.029  0.728825
 946.019  0.228572
 940.627  0.499199

In [8]:
scases = [s⁺(cases[i,1],cases[i,2]) for i in 1:size(cases,1)]

4-element Array{Complex{Float64},1}:
 0.194453+472.482im
 0.135588+473.515im
 0.385714+473.01im 
   0.2504+470.314im

In [9]:
Gs = [bigexp(lgamma(s)) for s in scases]

4-element Array{Complex{Float64},1}:
  4.94066e-324-1.97626e-323im
           0.0-4.94066e-324im
  -1.4822e-323+1.97626e-323im
 -3.16202e-322-7.11455e-322im

In [10]:
setprecision(80)do
ratios = Array{Float64}(length(scases),3)
for i in 1:length(scases)
    s = scases[i]
    G = bigexp(lgamma(s))
    est = est5(s)
    ratios[i,2] = abs(G)
    ratios[i,1] = abs(est[1]/est[2])
    ratios[i,3] = abs(est[1]*est[2])
end
ratios
end

4×3 Array{Float64,2}:
 1.4822e-323   1.97626e-323  1.4822e-323 
 4.94066e-324  4.94066e-324  4.94066e-324
 2.96439e-323  2.47033e-323  2.96439e-323
 7.85564e-322  7.80624e-322  7.85564e-322

This looks numerical.

Attempting to isolate the numerical issue, we first check gamma against lgamma (log gamma) and then check some functional relations.

In [11]:
@show gamma(scases[1]) ≈ bigexp(lgamma(scases[1]))
@show gamma(scases[1]) ≈ gamma(scases[1]+1)/scases[1]
for k in 1:5
    est = est5(scases[1]+k)
    den = prod([scases[1]+j for j in 0:(k-1)])
    @show k, abs(est[1]/est[2]) ≤ abs(gamma(scases[1]+k)) ≤ abs(est[1]*est[2])
    @show k, abs(est[1]/est[2]/den) ≤ abs(gamma(scases[1]+k)/den) ≤ abs(est[1]*est[2]/den)
end
nothing

gamma(scases[1]) ≈ bigexp(lgamma(scases[1])) = true
gamma(scases[1]) ≈ gamma(scases[1] + 1) / scases[1] = true
(k, abs(est[1] / est[2]) ≤ abs(gamma(scases[1] + k)) ≤ abs(est[1] * est[2])) = (1, false)
(k, abs((est[1] / est[2]) / den) ≤ abs(gamma(scases[1] + k) / den) ≤ abs((est[1] * est[2]) / den)) = (1, true)
(k, abs(est[1] / est[2]) ≤ abs(gamma(scases[1] + k)) ≤ abs(est[1] * est[2])) = (2, true)
(k, abs((est[1] / est[2]) / den) ≤ abs(gamma(scases[1] + k) / den) ≤ abs((est[1] * est[2]) / den)) = (2, true)
(k, abs(est[1] / est[2]) ≤ abs(gamma(scases[1] + k)) ≤ abs(est[1] * est[2])) = (3, true)
(k, abs((est[1] / est[2]) / den) ≤ abs(gamma(scases[1] + k) / den) ≤ abs((est[1] * est[2]) / den)) = (3, true)
(k, abs(est[1] / est[2]) ≤ abs(gamma(scases[1] + k)) ≤ abs(est[1] * est[2])) = (4, true)
(k, abs((est[1] / est[2]) / den) ≤ abs(gamma(scases[1] + k) / den) ≤ abs((est[1] * est[2]) / den)) = (4, true)
(k, abs(est[1] / est[2]) ≤ abs(gamma(scases[1] + k)) ≤ abs(est[1] * est[2])) = (5, true)

Which suggests that either the estimate or gamma itself is unreliable when $|\Gamma(s)| \le 2.$

In [13]:
@show g = gamma(scases[1])
for k in 1:5
    den = prod([scases[1]+j for j in 0:(k-1)])
    gk =  gamma(scases[1]+k)
    @show k, gk, gk/den≈g
end
nothing

g = gamma(scases[1]) = 5.0e-324 - 2.0e-323im
(k, gk, gk / den ≈ g) = (1, 8.29e-321 + 2.33e-321im, true)
(k, gk, gk / den ≈ g) = (2, -1.091095e-318 + 3.9186e-318im, true)
(k, gk, gk / den ≈ g) = (3, -1.853859954e-315 - 5.0692338e-316im, true)
(k, gk, gk / den ≈ g) = (4, 2.3358990366e-313 - 8.775340731e-313im, true)
(k, gk, gk / den ≈ g) = (5, 4.1559849034689e-310 + 1.0668615748881e-310im, true)


SpecialFunctions' gamma implementation seems to satisfy $\Gamma(z) \approx \frac{\Gamma(z+k)}{z\cdots z^{k-1}}$ which tends to implicate `est5`. We have

$\Gamma(z) = \sqrt{2\pi} \exp( (z-\frac{1}{2}) \log z - z + O_{\leq}( \frac{1}{12(|z| - 0.33)} )).$

In [15]:
@show g = est5(scases[1])
for k in 1:5
    den = prod([scases[1]+j for j in 0:(k-1)])
    gk =  est5(scases[1]+k)
    @show k, gk[1]/den, gk[1]/den≈g[1]
end
nothing    

g = est5(scases[1]) = (0.0 - 1.5e-323im, 1.0001765125430555)
(k, gk[1] / den, gk[1] / den ≈ g[1]) = (1, 5.0e-324 - 2.0e-323im, false)
(k, gk[1] / den, gk[1] / den ≈ g[1]) = (2, 5.0e-324 - 2.0e-323im, false)
(k, gk[1] / den, gk[1] / den ≈ g[1]) = (3, 5.0e-324 - 2.0e-323im, false)
(k, gk[1] / den, gk[1] / den ≈ g[1]) = (4, 5.0e-324 - 2.0e-323im, false)
(k, gk[1] / den, gk[1] / den ≈ g[1]) = (5, 5.0e-324 - 2.0e-323im, false)


Oddly, the $\Gamma(z) \approx \frac{\Gamma(z+k)}{z\cdots z^{k-1}}$ estimates match the `SpecialFunctions.gamma` estimate of $5.0e-324 - 2.0e-323i$ better than the straightforward estimation. This may be the root of the problem. Exploring further, 

In [22]:
s = scases[1]
@show G=lgamma(s)
@show est=log(2*π)/2 + (s-1/2)*log(s)-s
@show err=1/12/(abs(s)-0.33)
@show abs(G-est) ≤ err
@show abs(bigexp(G-est)) ≤ bigexp(err)
nothing

G = lgamma(s) = -743.1349912679964 + 2436.579604220866im
est = (log(2π) / 2 + (s - 1 / 2) * log(s)) - s = -743.1349913405842 + 2436.5797805945713im
err = (1 / 12) / (abs(s) - 0.33) = 0.00017649696654944592
abs(G - est) ≤ err = true
abs(bigexp(G - est)) ≤ bigexp(err) = true


This is fairly curious. It would appear that $\sqrt(2\pi)$ should be in the exponent. How can that make a difference?

In [23]:
oldest = √(2*π)*bigexp((s-0.5)*log(s)-s)
olderr = bigexp(1/(12*(abs(s)-0.33)))
@show oldest, bigexp(est)
@show olderr, bigexp(err)
nothing

(oldest, bigexp(est)) = (0.0 - 1.5e-323im, 5.0e-324 - 2.0e-323im)
(olderr, bigexp(err)) = (1.0001765125430555, 1.0001765125430555)


(1.0001765125430555, 1.0001765125430555)

So, what is happening here? Is the $\log(\sqrt{2\pi})$ figure making a difference in the exponentiation?

In [30]:
arg = (s-0.5)*log(s)-s
@show bigexp(arg)
@show a1=abs(bigexp(arg+log(2*π)/2))
@show a2=exp(real(arg))*√(2*π)
@show a1/a2
nothing

bigexp(arg) = 0.0 - 5.0e-324im
a1 = abs(bigexp(arg + log(2π) / 2)) = 2.0e-323
a2 = exp(real(arg)) * √(2π) = 1.5e-323
a1 / a2 = 1.3333333333333333


There is obviously a big "discontinuity" in bigexp. For example, if I do a multiprecision version of the above:

In [31]:
setprecision(80)do
    s′=bigify(s)
    arg = (s′-0.5)*log(s′)-s′
    @show bigexp(arg)
    @show a1=abs(bigexp(arg+log(2*π)/2))
    @show a2=exp(real(arg))*√(2*π)
    @show a1/a2
end
nothing

bigexp(arg) = 1.971662504549982811315041e-324 - 6.9966551144759609350823914e-324im
a1 = abs(bigexp(arg + log(2π) / 2)) = 1.8221073146872046898256305e-323
a2 = exp(real(arg)) * √(2π) = 1.8221073146872046318714174e-323
a1 / a2 = 1.0000000000000000318061467


and in straight single precision

In [32]:
arg = (s-0.5)*log(s)-s
@show exp(arg)
@show a1=abs(exp(arg+log(2*π)/2))
@show a2=exp(real(arg))*√(2*π)
@show a1/a2
nothing


exp(arg) = 0.0 - 5.0e-324im
a1 = abs(exp(arg + log(2π) / 2)) = 2.0e-323
a2 = exp(real(arg)) * √(2π) = 1.5e-323
a1 / a2 = 1.3333333333333333


So it is a precision problem in exponentiation. Can it be predetermined and avoided?

In [50]:
@show real(arg)
@show real(arg) + log(2*π)/2
@show exp(real(arg) + log(2*π)/2)
@show exp(real(arg))*exp(log(2*π)/2)
nothing

real(arg) = -744.0539298737889
real(arg) + log(2π) / 2 = -743.1349913405842
exp(real(arg) + log(2π) / 2) = 2.0e-323
exp(real(arg)) * exp(log(2π) / 2) = 1.5e-323


In [56]:
@show precision(real(arg))
setprecision(54)do
    arg′=bigify(arg)
    @show precision(real(arg′))
    @show real(arg′)
    @show real(arg′) + log(2*π)/2
    @show exp(real(arg′) + log(2*π)/2)
    @show exp(real(arg′))*exp(log(2*π)/2)
end
nothing

precision(real(arg)) = 53
precision(real(arg′)) = 54
real(arg′) = -7.44053929873788888e+02
real(arg′) + log(2π) / 2 = -7.43134991340584236e+02
exp(real(arg′) + log(2π) / 2) = 1.8221073146872217e-323
exp(real(arg′)) * exp(log(2π) / 2) = 1.82210731468725812e-323


In [65]:
setprecision(54)do
    for s in scases
        G = bigexp(bigify(lgamma(s)))
        est = est5(bigify(s))
        @show 1/est[2] ≤ abs(G/est[1]) ≤ est[2]
    end
end
nothing

1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true


It appears that the `Float64` exponentiation method is flawed in comparision with the `BigFloat` method, even when `BigFloat` precision is only 1 bit more than `Float64`'s. Is this true when precision is the same?

In [67]:
setprecision(53)do
    for s in scases
        G = bigexp(bigify(lgamma(s)))
        est = est5(bigify(s))
        @show 1/est[2] ≤ abs(G/est[1]) ≤ est[2]
    end
end
nothing

1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true


It does. This seems to be the root of the problem.

In [68]:
setprecision(53)do
    for s in scases
        G = big(e)^(lgamma(s))
        est = √(2*π)*big(e)^((s-0.5)*log(s)-s),big(e)^(1/(12*(abs(s)-0.33)))
        @show 1/est[2] ≤ abs(G/est[1]) ≤ est[2]
    end
end
nothing

1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true
1 / est[2] ≤ abs(G / est[1]) ≤ est[2] = true


In [91]:
tmp = Vector{Bool}(size(r5,1))
setprecision(53)do
    for i in 1:size(r5,1)
        s = s⁺(r5[i,2],r5[i,3])
        G = big(e)^(lgamma(s))
        est = √(2*π)*big(e)^((s-0.5)*log(s)-s),big(e)^(1/(12*(abs(s)-0.33)))
        tmp[i] = 1/est[2] ≤ abs(G/est[1]) ≤ est[2]
    end
end
all(tmp)


true